# ES uberh3-hex at res5 for UF-DEMO (ogrID = 444)organization_id == 444

In [1]:
import sqlalchemy
import geopandas as gp
import pandas as pd
from shapely.geometry import Point
%load_ext autotime
import h3
import numpy as np
from elasticsearch import Elasticsearch, helpers
import json


# DEV
es = Elasticsearch(['https://3d6a9dd50c7c49c9ab5d23b6891bc03e.us-central1.gcp.cloud.es.io:9243'], 
                    http_auth=('elastic', 'WMzYk5RXyzE7MRShwPVwHzPX'))

# PROD
# browser LOGIN-link: https://003712d383a54253aa3b4feee5c866d6.us-central1.gcp.cloud.es.io:9243/
es_prod = Elasticsearch(["https://e18a827982904fcb8ab47f7bdc8a0792.us-central1.gcp.cloud.es.io:9243"], 
                        http_auth=('elastic', 'xQKsNu7BcFUbww2uoZvu6lyc'), timeout=30)

es.ping(), es_prod.ping()

/home/nhat/anaconda3/envs/py39/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.0-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


(True, True)

time: 135 ms (started: 2022-03-02 10:22:41 -05:00)


In [2]:
def download_ES(index_name, es_instance = es):
    print('Start downloading ES index ', index_name)
#     match_query = { "query": {"match_all": {}} }
    match_query = {
        "query": {"match_all": {}},
        "_source": {
            "includes": [ 'id', 'mailingAddress']
        },
    }
    res_gen = helpers.scan(es_instance, query= match_query, index= index_name) # C x faster than (match_all without `includes`)
    data = []
    for record in res_gen:
        record['_source']['row_id'] = record['_id']
        data.append(record['_source'])
    print('Completed downloading ES index ', index_name)
    return pd.DataFrame(data)


time: 639 µs (started: 2022-03-02 10:22:57 -05:00)


In [2]:
pg_driver = 'postgresql+psycopg2'
pg_user = 'boss_user'
pg_pass = 'passDEV9g47uibjn2ijovZNEW' # boss dev
pg_host = 'dev-boss.db.ready.net' # boss dev
pg_port = '5432'
pg_db = 'boss_db_dev'

engine = sqlalchemy.create_engine(f"{pg_driver}://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}") 

cols = [
    'distinct u.id',
    'op.organization_id',
    'op.role',
    'a.lat',
    'a.lon']
select_cols = ','.join(cols)

# left outer join = include users not found in ogr_permission table
# inner join = only include users found in ogr_permission table
sql_hex = f"""
    select {select_cols}
    from users u
    inner join organization_permissions op on (u.id = op.user_id)
    inner join addresses a on (u.mailing_address_id = a.id)
    """

users = pd.read_sql_query(sql_hex, engine)
users.shape, len(ufdemo.id.unique()) # ((484873, 5), 484855)

((527041, 5), 527023)

time: 1min 9s (started: 2022-02-22 20:18:42 -05:00)


### UF demo org_id = 444

In [28]:
# drop rows with empty lat,lon [for now]
ufdemo = users[users.lat.notnull() | users.lon.notnull()] # (72049, 3)
ufdemo = ufdemo[ufdemo.role != 'admin']
ufdemo = ufdemo.loc[ufdemo.organization_id == 444][['id', 'lat', 'lon']]
ufdemo.shape

(72049, 3)

time: 114 ms (started: 2022-02-15 11:22:21 -05:00)


In [ ]:
ufdemo['hexCode7'] = ufdemo.apply(lambda row: h3.geo_to_h3(row['lat'], row['lon'], 7), axis=1)
ufdemo['hexCode8'] = ufdemo.apply(lambda row: h3.geo_to_h3(row['lat'], row['lon'], 8), axis=1)

### UF production in PRODUCTION-db -- organization_id = 25

- on local machine: cd local
- start cloud_sql_proxy: $ ./cloud_sql_proxy -instances=boss-api-289815:us-central1:boss-datastore=tcp:5432 -credential_file=boss-api-289815-bc672ce4aaf5.json
- Then: can access prod-dv using connection-string below


In [18]:
# CONNECT TO PG-PROD SOURCE
pg_driver = 'postgresql+psycopg2'
pg_user = 'postgres'
pg_pass = 'qweuyf7web392dsa' 
pg_host = '127.0.0.1'
pg_port = '5432'
pg_db = 'boss-db'

engine = sqlalchemy.create_engine(f"{pg_driver}://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}") 

cols = [
    'distinct u.id',
    'op.organization_id',
    'op.role',
    'a.lat',
    'a.lon']
select_cols = ','.join(cols)

# NOTE: usage of inner join = only include users found in ogr_permission table
# instead of using left outer join = include users not found in ogr_permission table
sql_hex = f"""
    select {select_cols}
    from users u
    inner join organization_permissions op on (u.id = op.user_id)
    inner join addresses a on (u.mailing_address_id = a.id)
    """

pg_prod_users = pd.read_sql_query(sql_hex, engine)
print(pg_prod_users.shape)
# Remove users without lat, lon
pg_prod_null_coords = pg_prod_users[pg_prod_users.lat.isnull() | pg_prod_users.lon.isnull()]
print(pg_prod_null_coords.shape) # 120
print(sorted(set(pg_prod_null_coords.id)))
pg_prod_users = pg_prod_users[pg_prod_users.lat.notnull() | pg_prod_users.lon.notnull()] 

pg_prod_users.shape, len(pg_prod_users.id.unique()), 

(206480, 5)
(120, 5)
[2, 6, 12, 13, 15, 18, 25, 32, 38, 39, 40, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 57, 58, 59, 60, 63, 64, 75, 76, 77, 91, 99, 102, 106, 110, 123, 126, 145, 164, 177, 187, 207, 211, 215, 216, 217, 238, 249, 252, 254, 257, 261, 359, 360, 361, 432, 435, 641, 659, 660, 663, 1510, 1511, 1512, 1513, 1514, 503958, 724009, 725728, 987454, 987455, 987456, 987457, 987458, 987459, 987477, 987478, 987479, 987480, 987482, 987483, 987484, 987485, 987486, 987487, 987492, 987493, 987494, 987495, 987497, 987498, 987499, 987500, 987501, 987504, 987505, 987506, 1170328, 1170329, 1170337, 1170338, 1170339, 1170340, 1170341, 1170342, 1170343, 1170344, 1170345, 1170346, 1170347, 1170350, 1170351, 1170352, 1170354, 1170355, 1170357, 1170358, 1170359]


((206360, 5), 206360)

time: 1.53 s (started: 2022-03-02 10:38:19 -05:00)


In [19]:
pg_prod_users[pg_prod_users.role == 'admin'].shape

(0, 5)

time: 12 ms (started: 2022-03-02 10:38:22 -05:00)


In [20]:
pg_prod_users = pg_prod_users[pg_prod_users.role != 'admin']
# UF PROD organization
ufprod = pg_prod_users.loc[pg_prod_users.organization_id == 25][['id', 'lat', 'lon']]
ufprod.shape # (203104, 3)

(203104, 3)

time: 26.3 ms (started: 2022-03-02 10:38:28 -05:00)


In [ ]:
ufprod['hexCode7'] = ufprod.apply(lambda row: h3.geo_to_h3(row['lat'], row['lon'], 7), axis=1)
ufprod['hexCode8'] = ufprod.apply(lambda row: h3.geo_to_h3(row['lat'], row['lon'], 8), axis=1)
ufprod['hexCode'] = ufprod.apply(lambda row: h3.geo_to_h3(row['lat'], row['lon'], 15), axis=1)


# ES: 2 new mappings: hexCode7 and hexCode8 inside mailing address, type keyword

In [35]:
# def update_bossdata_index(es, index_name):
#     update_request_body = {
#         "properties":{
#             "mailingAddress": {
#                 "type": "nested",
#                 "properties": {
#                     "hexCode7": {  "type": "keyword" },
#                     "hexCode8": {  "type": "keyword" },
#                 }
#             },
            
#         }
#     }
#     es.indices.put_mapping(index = index_name, body = update_request_body)
#     print('updated index ', index_name)

# update_bossdata_index(es, 'users1.0.2')

updated index  users1.0.2
time: 164 ms (started: 2022-02-15 11:28:38 -05:00)


# ES download/upload

## es_prod (index alias: usersAlias, most likely pointing to usersnew (not the older index: users)

In [79]:
# download_ES('usershexcode', es).shape # (80000, 3)
# download_ES('users', es).shape # (401799, 3)
# # usersAlias is pointing to users1.0.2 in ES-dev
# download_ES('usersAlias', es).shape # (503310, 3)
# download_ES('users1.0.2', es).shape # (503152, 3)
# download_ES('usersnew', es).shape # (513217, 3)

Start downloading ES index  usersnew
Completed downloading ES index  usersnew


(513217, 3)

time: 1min 41s (started: 2022-02-23 00:04:24 -05:00)


In [16]:
users_ES = download_ES('usersAlias', es_prod)
# users_ES_usersnew = download_ES('usersnew', es_prod)
# users_ES.shape, users_ES_usersnew.shape # ((433684, 3), (433684, 3))
users_ES.shape

Start downloading ES index  usersAlias
Completed downloading ES index  usersAlias
time: 47.5 s (started: 2022-02-23 12:18:34 -05:00)


In [23]:
len(users), len(ufprod)

(272957, 272858)

time: 1.87 ms (started: 2022-02-23 12:27:25 -05:00)


In [85]:
len(users_ES), len(users_ES.id.unique())

# e.g. https://32a112996b92429ca3c682984cdf5fc4.us-central1.gcp.cloud.es.io:9243/app/discover?#/?_a=(columns:!(_id,id,email,mailingAddressId,mailingAddress),filters:!(),index:c75748d0-89cd-11ec-a00d-7ff105ba8ac8,interval:auto,query:(language:kuery,query:'id%20:%201768291'),sort:!())&_g=(filters:!(),refreshInterval:(pause:!t,value:0),time:(from:now-15m,to:now))
# for user with id = 1768291, there are 3 records: _id = [50032, 275963, 48880]

(433684, 433684)

time: 12.1 ms (started: 2022-02-23 00:14:52 -05:00)


In [33]:
print(set(ufprod.id) - set(users_ES.id))

{518464, 715686, 521513, 505616, 520532, 505781, 521366, 522964, 521402, 518332, 521119}
time: 108 ms (started: 2022-02-23 12:32:12 -05:00)


In [89]:
merged_users = ufprod.set_index('id').join(users_ES.set_index('id'), how='inner')
merged_users = merged_users.rename(columns={'_id': 'row_id'})
len(merged_users), len(set(merged_users.index)), len(set(merged_users.row_id))

(269591, 269591, 269591)

time: 563 ms (started: 2022-02-23 00:17:05 -05:00)


In [90]:

merged_users.loc[merged_users.index.duplicated(keep=False), :]

,lat,lon,hexCode7,hexCode8,hexCode,mailingAddress,row_id
id,,,,,,,


time: 8.38 ms (started: 2022-02-23 00:17:20 -05:00)


In [95]:
def upload_ES(idCol, df, index_name, es_instance = es):
    # BIG NOTE: idCol name should not be: _id (an ES reserved keyword)
    # updating_dict: a dict of all the data to be updated
    updating_dict = df.drop(columns=idCol).to_json(orient = "records")
    updating_dict = json.loads(updating_dict)

    # create an actiondf for all actions
    actiondf = df[[idCol]].copy()
    actiondf['_op_type'] = 'update'
    actiondf['_index'] = index_name
    actiondf['_id'] = actiondf[idCol] # still need 
    actiondf['doc_as_upsert'] = True
    actiondf['doc'] = updating_dict 
    actiondf.drop(idCol, axis = 1, inplace = True) # # NOTE: we are not keeping the idCol -- row_id in this case

    udf = actiondf.to_json(orient = 'records')
    # all actions
    action_list = json.loads(udf) 
    print(f"Start uploading {len(df)} records to {index_name}")
    helpers.bulk(es_instance, action_list)
    print(f"Completed uploading {len(df)} records to {index_name}")


time: 824 µs (started: 2022-02-23 00:20:39 -05:00)


In [91]:
merged_users.mailingAddress = merged_users.apply(lambda row:
        {'lat': row.lat, 'lon': row.lon, 'hexCode': row.hexCode, 'hexCode7': row.hexCode7, 'hexCode8': row.hexCode8 }, axis = 1)

time: 6.46 s (started: 2022-02-23 00:17:58 -05:00)


### SELECT ES df, and Upload to ES (dev instance, or prod instance)

In [93]:
es_df = merged_users[['row_id', 'mailingAddress']].copy()
print(es_df.shape)
es_df.mailingAddress.head(5).values

(269591, 2)


array([{'lat': 40.76010998, 'lon': -94.10828, 'hexCode': '8f26099a671b4db', 'hexCode7': '8726099a6ffffff', 'hexCode8': '8826099a67fffff'},
       {'lat': 40.75716001, 'lon': -94.95295003, 'hexCode': '8f260832a3a9989', 'hexCode7': '87260832affffff', 'hexCode8': '88260832a3fffff'},
       {'lat': 39.44182001, 'lon': -94.89959997, 'hexCode': '8f2656c8a671a62', 'hexCode7': '872656c8affffff', 'hexCode8': '882656c8a7fffff'},
       {'lat': 39.345422, 'lon': -94.22149604, 'hexCode': '8f26568dd350434', 'hexCode7': '8726568ddffffff', 'hexCode8': '8826568dd3fffff'},
       {'lat': 39.342726, 'lon': -94.22298098, 'hexCode': '8f26568dd20b68d', 'hexCode7': '8726568ddffffff', 'hexCode8': '8826568dd3fffff'}],
      dtype=object)

time: 28.7 ms (started: 2022-02-23 00:19:06 -05:00)


In [112]:
es_df.sample(5)

,row_id,mailingAddress
id,,
917684,U3E2JX4B9cn-h0Vl-VKZ,"{'lat': 39.2803398, 'lon': -93.9828406, 'hexCo..."
977564,omlCJn4B7Ve0ZrKVfbBH,"{'lat': 39.2427349, 'lon': -94.4728748, 'hexCo..."
61902,57259,"{'lat': 39.7370493, 'lon': -94.2389126, 'hexCo..."
761094,gXCaIn4B9cn-h0VlhQZJ,"{'lat': 39.7035062, 'lon': -94.2815917, 'hexCo..."
754887,eGeAIn4B7Ve0ZrKVCvKP,"{'lat': 40.0483045, 'lon': -95.3214662, 'hexCo..."


time: 11.7 ms (started: 2022-02-23 00:38:24 -05:00)


In [109]:
es_df.shape

(269591, 2)

time: 1.44 ms (started: 2022-02-23 00:28:09 -05:00)


In [ ]:
es_df.isnull().sum().sum()

In [110]:
### quick sample upload for a random user
sample = es_df.loc[es_df.index == 20639]
sample
upload_ES('row_id', sample, 'usersAlias', es_prod)
sample.mailingAddress.values


Start uploading 1 records to usersAlias
Completed uploading 1 records to usersAlias


array([{'lat': 39.342726, 'lon': -94.22298098, 'hexCode': '8f26568dd20b68d', 'hexCode7': '8726568ddffffff', 'hexCode8': '8826568dd3fffff'}],
      dtype=object)

time: 42.8 ms (started: 2022-02-23 00:30:46 -05:00)


In [111]:
# ACTUAL UPLOAD
# upload_ES('row_id', es_df, 'usersAlias', es) # ES DEV instance
# upload_ES('row_id', es_df, 'usersAlias', es_prod)

Start uploading 269591 records to usersAlias
Completed uploading 269591 records to usersAlias
time: 3min 6s (started: 2022-02-23 00:33:00 -05:00)


### EXTRA: testing upload certain fields of nested object 

In [ ]:
def create_ES_index(es, index_name):
    create_request_body = {
       "settings": {
        "refresh_interval": "1s",
        "number_of_shards": 1,
        "number_of_replicas": 0
      },
        "mappings":{ 
            "properties":{
                "id":    { "type": "text"  },
                "mailingAddress": {
                    "type": "nested",
                    "properties": {
                        "lat" : { "type": "text" },
                        "hexCode": {  "type": "keyword" },
                    }
                },
                "size": { "type" : "integer" },
            }
        }
    } 

    es.indices.create(index = index_name, body = create_request_body)
    print('created index ', index_name)

# create_ES_index(es, 'nhat_test1')

In [207]:
test_upload = merged_users.loc[:, ['row_id', 'mailingAddress', 'hexCode']]
test_upload.mailingAddress = test_upload.apply(lambda row: 
                    {'hexCode': row.hexCode}, axis=1)
test_upload.head()

,row_id,mailingAddress,hexCode
id,,,
1686295,281337,{'hexCode': '8526095bfffffff'},8526095bfffffff
1719266,217625,{'hexCode': '85260ba7fffffff'},85260ba7fffffff
1719267,76430,{'hexCode': '85260ba7fffffff'},85260ba7fffffff
1719268,216555,{'hexCode': '85265693fffffff'},85265693fffffff
1719269,215404,{'hexCode': '85260967fffffff'},85260967fffffff


time: 473 ms (started: 2022-02-15 00:33:50 -05:00)


In [210]:
test_upload['size'] = 10
test_upload['newField'] = 11

upload_ES('row_id', test_upload, 'nhat_test1')

Start uploading 67639 records to nhat_test1
Completed uploading 67639 records to nhat_test1
time: 7.19 s (started: 2022-02-15 00:34:56 -05:00)


In [109]:
# more fields inside nested object
test_upload.mailingAddress = test_upload.apply(lambda row:
            {'lat': 'Unknow', 'hexCode': row.hexCode}, axis = 1)

upload_ES('row_id', test_upload, 'nhat_test')

time: 1.02 s (started: 2022-02-14 23:36:15 -05:00)


In [214]:
# fewer field inside nested object
test_upload.mailingAddress = test_upload.apply(lambda row:
            {'hexCode': 'justkidding'}, axis = 1)

upload_ES('row_id', test_upload, 'nhat_test')

Start uploading 67639 records to nhat_test
Completed uploading 67639 records to nhat_test
time: 13.3 s (started: 2022-02-15 00:40:22 -05:00)
